In [ ]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

!pip install transformers
!pip install bert-tensorflow
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 
model_save_name = 'alberto_hodi.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 

Mounted at /content/gdrive


#Training misogyny

In [ ]:
data = pd.read_excel('haspeedee_train.xlsx')

In [ ]:
data

,anonymized_tweet_id,anonymized_text,label,dataset,data_type
0,217874450618134,@unknown Con tutte le denunce che si sta becca...,1,training_politics,train
1,360042217507605,#PRESCRIZIONE : I (t)re magi hanno trovato l’a...,0,training_politics,train
2,817917484817935,Il #m5s ha votato contro le #unionicivili addu...,1,training_politics,train
3,145836038456701,Che cosa cambia questa #legge? Caro @667517147...,0,training_politics,train
4,599642342348521,@800991101440753 Spero rientrino 2000 insegnan...,0,training_politics,train
...,...,...,...,...,...
5035,221538539806447,Sportello telematico attivo al Comune di Santa...,0,training_politics,train
5036,715670659156960,#Covid. #Draghi non fare come #Conte. La #poli...,0,training_politics,train
5037,512309888296620,"Cina, «128mila euro a neonato»: la proposta pe...",0,training_politics,train
5038,887061502405620,@401560935985106 @507718538254873 @28429789767...,1,training_politics,train


In [ ]:
data = data.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset in traning and validation(test)
X_train, X_val, Y_train, Y_val = train_test_split(
    data.index.values,
    data.label.values,
    test_size=0.06,
    random_state=17,
    stratify=data.label.values
)

In [ ]:
# Check datasets composition
data['data_type'] = ['not_set'] * data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.groupby(['label', 'data_type']).count()

anonymized_tweet_id  anonymized_text  dataset
label data_type                                               
0     train                     2923             2923     2923
      val                        187              187      187
1     train                     1814             1814     1814
      val                        116              116      116

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset


tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", use_fast=False)

In [ ]:
#change column name
data = data.rename(columns={'anonymized_text' : 'text'})

In [ ]:
# Encode training dataset using the tokenizer
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True, 
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Encode validation dataset using the tokenizer
encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Extract IDs, attention masks and labels from training dataset
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type == 'train'].label.values)
labels_train

tensor([1, 0, 1,  ..., 0, 1, 0])

In [ ]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type == 'val'].label.values)

In [ ]:
# Create train and validation dataset from extracted features
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
print("Train dataset length: {}\nValidation dataset length: {}".format(len(dataset_train), len(dataset_val)))

Train dataset length: 4737
Validation dataset length: 303


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Define the size of each batch
batch_size = 16

# Load training dataset
dataloader_train= DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size)

# Load valuation dataset
dataloader_val= DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

Some weights of the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define model optimizer -> Adam
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, 
    eps=1e-8
)
# Define model scheduler
epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import random

# Define random seeds
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# Define processor type for torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Returns the F1 score computed on the predictions
def f1_score_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat, preds_flat, average='macro')



# Returns the precision and recall score computed on the predictions
def prec_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return precision_score(labels_flat, preds_flat, average='macro')

def recall_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return recall_score(labels_flat, preds_flat, average='macro')



In [ ]:
# Evaluates the model using the validation set
def evaluate(dataloader_val):
  model.eval()
  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in dataloader_val:
      batch = tuple(b.to(device) for b in batch)
      inputs = {'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2],
        }

      with torch.no_grad():
          outputs = model(**inputs)

      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)

  loss_val_avg = loss_val_total / len(dataloader_val)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs + 1)):

    model.train()  # model is training

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()  # to backpropagate

        torch.nn.utils.clip_grad_norm_(model.parameters(),
                                      1.0)  # prevents the gradient from being too small or too big

        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    
    torch.save(model, path)
    tqdm.write(f'\nEpoch {epoch}/{epochs}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')  # make sure that model is still training

    val_loss, predictions, true_vals = evaluate(dataloader_val)  # to check overtraining (or overfitting)
    val_f1 = f1_score_func(predictions, true_vals)
    val_prec = prec_func(predictions, true_vals)
    val_recall = recall_func(predictions, true_vals)

    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score(weighted) : {val_f1}')
    tqdm.write(f'Prec Score(weighted) : {val_prec}')
    tqdm.write(f'Recall Score(weighted) : {val_recall}')


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 1/4
Training loss: 0.36340932112801755
Validation loss: 0.25048830869950744
F1 Score(weighted) : 0.8943336370692856
Prec Score(weighted) : 0.889186200796108
Recall Score(weighted) : 0.9056564632122441


Epoch 2:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 2/4
Training loss: 0.21213836737152703
Validation loss: 0.2450219426108034
F1 Score(weighted) : 0.9100520643039824
Prec Score(weighted) : 0.9068019201159315
Recall Score(weighted) : 0.9141158030610363


Epoch 3:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 3/4
Training loss: 0.15193000763046371
Validation loss: 0.3018527997559623
F1 Score(weighted) : 0.8992143143889431
Prec Score(weighted) : 0.8974470222871758
Recall Score(weighted) : 0.9011847685782777


Epoch 4:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 4/4
Training loss: 0.11706255062997517
Validation loss: 0.3337841548613812
F1 Score(weighted) : 0.9072543617998163
Prec Score(weighted) : 0.9024695776664281
Recall Score(weighted) : 0.9147151023418771


In [ ]:
model.save_pretrained('content/gdrive/MyDrive/alberto_haspeede')

In [ ]:
results = []
for i, prediction in enumerate(predictions):
  predicted = np.argmax(prediction)
  # print(f"index: {i} -- prediction: {predicted}")
  results.append(predicted)

In [ ]:
hate = []
for prediction in results:
  hate.append(prediction)

In [ ]:
data

,anonymized_tweet_id,text,label,dataset,data_type
0,217874450618134,@unknown Con tutte le denunce che si sta becca...,1,training_politics,train
1,360042217507605,#PRESCRIZIONE : I (t)re magi hanno trovato l’a...,0,training_politics,train
2,817917484817935,Il #m5s ha votato contro le #unionicivili addu...,1,training_politics,train
3,145836038456701,Che cosa cambia questa #legge? Caro @667517147...,0,training_politics,train
4,599642342348521,@800991101440753 Spero rientrino 2000 insegnan...,0,training_politics,train
...,...,...,...,...,...
5035,221538539806447,Sportello telematico attivo al Comune di Santa...,0,training_politics,train
5036,715670659156960,#Covid. #Draghi non fare come #Conte. La #poli...,0,training_politics,train
5037,512309888296620,"Cina, «128mila euro a neonato»: la proposta pe...",0,training_politics,train
5038,887061502405620,@401560935985106 @507718538254873 @28429789767...,1,training_politics,train


#Preds sul val

In [ ]:
data_val = data.loc[data['data_type'] == 'val']

In [ ]:
data_val

,anonymized_tweet_id,text,label,dataset,data_type
3,172580609652325,La #Lega e il #m5s stanno dando a bere all'opi...,0,training_politics,val
28,825279779951638,Il decreto #salvini crea insicurezza e disuman...,0,training_politics,val
42,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,training_politics,val
48,327244336972419,#ottoemezzo. #Bongiorno ci racconta che il gov...,0,training_politics,val
51,897329723892923,#ottoemezzo #cacciari spiega alla #bongiorno l...,0,training_politics,val
...,...,...,...,...,...
5557,324165730844406,".@738958761853559 , ma vafangul’\n\nmessaggio ...",1,training_politics,val
5570,871453511657474,#ENRICOLETTA É DIARREA BERLUSCONICCHIA CHE POR...,1,training_politics,val
5580,67223505911646,"Ultima Ora: Webuild, nuovi ordini per 2,6 mili...",0,training_politics,val
5588,517002584005250,#14maggio #PapaFrancesco Occorrono politiche f...,0,training_politics,val


In [ ]:
data_val['hate'] = hate

<ipython-input-51-0caca47192c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val['hate'] = hate


In [ ]:
data_val

,anonymized_tweet_id,text,label,dataset,data_type,hate
3,172580609652325,La #Lega e il #m5s stanno dando a bere all'opi...,0,training_politics,val,0
28,825279779951638,Il decreto #salvini crea insicurezza e disuman...,0,training_politics,val,0
42,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,training_politics,val,1
48,327244336972419,#ottoemezzo. #Bongiorno ci racconta che il gov...,0,training_politics,val,0
51,897329723892923,#ottoemezzo #cacciari spiega alla #bongiorno l...,0,training_politics,val,0
...,...,...,...,...,...,...
5557,324165730844406,".@738958761853559 , ma vafangul’\n\nmessaggio ...",1,training_politics,val,0
5570,871453511657474,#ENRICOLETTA É DIARREA BERLUSCONICCHIA CHE POR...,1,training_politics,val,0
5580,67223505911646,"Ultima Ora: Webuild, nuovi ordini per 2,6 mili...",0,training_politics,val,0
5588,517002584005250,#14maggio #PapaFrancesco Occorrono politiche f...,0,training_politics,val,0


In [ ]:
data_val.to_excel('HaSpeeDe_preds.xlsx', index=False)

#Preds sul test

In [ ]:
#Load test set
data_test = pd.read_excel('haspeedee_test.xlsx')
data_test.head()

,anonymized_tweet_id,anonymized_text,label,dataset,data_type
0,172580609652325,La #Lega e il #m5s stanno dando a bere all'opi...,0,training_politics,val
1,825279779951638,Il decreto #salvini crea insicurezza e disuman...,0,training_politics,val
2,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,training_politics,val
3,327244336972419,#ottoemezzo. #Bongiorno ci racconta che il gov...,0,training_politics,val
4,897329723892923,#ottoemezzo #cacciari spiega alla #bongiorno l...,0,training_politics,val


In [ ]:
# Encode validation dataset using the tokenizer
encoded_data_test = tokenizer.batch_encode_plus(
    data_test.anonymized_text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

dataset_test = TensorDataset(input_ids_test, attention_masks_test)
print("Test dataset length: {}".format(len(dataset_test)))

Test dataset length: 560


In [ ]:
from torch.utils.data import DataLoader
dataloader_test = DataLoader(dataset_test)

In [ ]:
# Evaluates the model using the validation set
def predict(dataset_test):
    predictions = []

    for row in dataset_test:
      row = tuple(r.to(device) for r in row)
      inputs = {'input_ids': row[0],
        'attention_mask': row[1]
        }

      with torch.no_grad():
          outputs = model(**inputs)

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictions.append(logits)

    return predictions

# Predict values for test dataset
predictions = predict(dataloader_test)

In [ ]:
print(len(predictions))
results = []
for i, prediction in enumerate(predictions):
  predicted = np.argmax(prediction, axis=1)[0]
  # print(f"index: {i} -- prediction: {predicted}")
  results.append(predicted)

print(results)
print(results.count(0))
print(results.count(1))

560
[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,

In [ ]:
misogynous = []
for prediction in results:
  misogynous.append(prediction)

In [ ]:
#Predict misogyny
data_test['preds'] = misogynous


In [ ]:
#Save the predicted labels in a csv file 
data_test.to_excel('preds_haspeedee_test.xlsx', index=False)

#Testing 

In [ ]:
import pandas as pd
import sklearn
from sklearn.metrics import classification_report

In [ ]:
hs = pd.read_excel('preds_haspeedee_test.xlsx')

In [ ]:
hs

,anonymized_tweet_id,anonymized_text,label,dataset,data_type,preds
0,172580609652325,La #Lega e il #m5s stanno dando a bere all'opi...,0,training_politics,val,0
1,825279779951638,Il decreto #salvini crea insicurezza e disuman...,0,training_politics,val,0
2,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,training_politics,val,0
3,327244336972419,#ottoemezzo. #Bongiorno ci racconta che il gov...,0,training_politics,val,0
4,897329723892923,#ottoemezzo #cacciari spiega alla #bongiorno l...,0,training_politics,val,0
...,...,...,...,...,...,...
555,324165730844406,".@738958761853559 , ma vafangul’\n\nmessaggio ...",1,training_politics,val,1
556,871453511657474,#ENRICOLETTA É DIARREA BERLUSCONICCHIA CHE POR...,1,training_politics,val,1
557,67223505911646,"Ultima Ora: Webuild, nuovi ordini per 2,6 mili...",0,training_politics,val,0
558,517002584005250,#14maggio #PapaFrancesco Occorrono politiche f...,0,training_politics,val,0


In [ ]:
y_true = hs['label'].to_numpy()
y_preds = hs['preds'].to_numpy()
print(sklearn.metrics.classification_report(y_true, y_preds))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       346
           1       0.88      0.90      0.89       214

    accuracy                           0.91       560
   macro avg       0.91      0.91      0.91       560
weighted avg       0.91      0.91      0.91       560



#Data Augmentation

In [ ]:
data = pd.read_excel('haspeedee_train.xlsx')

In [ ]:
data.label.value_counts()

0    3110
1    1930
Name: label, dtype: int64

In [ ]:
import imblearn

In [ ]:
# summarize class distribution
counter = Counter(y)
print(counter)

In [ ]:
data_pos = data[data['label'] == 1]

In [ ]:
data_pos

,anonymized_tweet_id,anonymized_text,label,dataset,data_type
0,217874450618134,@unknown Con tutte le denunce che si sta becca...,1,training_politics,train
2,817917484817935,Il #m5s ha votato contro le #unionicivili addu...,1,training_politics,train
7,170378636015491,Ma se la #carfagna e #crosetto sono più a #sin...,1,training_politics,train
12,906842219275344,Nemmeno la tradizionale conferenza stampa di f...,1,training_politics,train
20,919547819000519,“#Bari: Con una ruspa rapinano un furgone port...,1,training_politics,train
...,...,...,...,...,...
5010,252759665270019,#Lega e #salvinisciacallo senza #VERGOGNA e s...,1,training_politics,train
5011,215800859616571,Queste sono le vaccate che solo il QI basso de...,1,training_politics,train
5012,535723231606582,"Sfiducia. Speranza: “Il nemico è il virus, non...",1,training_politics,train
5013,518258878031869,Ah....\nMi ricorda i collage che facevo quando...,1,training_politics,train


#Error Analysis

In [ ]:
#Error analysis
error = hs[hs['label'] != hs['preds']]
error.label.value_counts() #num di FP e FN è bilanciato, + 6 FP

0    27
1    21
Name: label, dtype: int64

In [ ]:
import nltk
text_list = error['anonymized_text'].tolist()
#remove stopwords
nltk.download('stopwords')
nltk.download('punkt')

stopwords = set(nltk.corpus.stopwords.words('italian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def convert(lst):
    return ' '.join(lst).split()
     
print(convert(text_list))

['"Paolo', 'Polidori"', 'dovrebbe', 'dimettersi', 'per', 'manifesta', 'incapacità:', 'invece', 'di', 'provvedere', 'nei', 'modi', 'consentiti', 'dalla', '#legge', 'e', 'dalla', 'sua', 'funzione', 'di', '#vicesindaco', 'risolvendo', 'la', 'situazione', 'di', 'disagio,', 'ha', 'preferito', 'agire', 'da', 'vandalo.', 'Non', 'merita', 'di', 'stare', 'in', 'un', 'Consiglio', 'Comunale.', '#Trieste', '@868843245148057', 'W', 'il', '#m5s', 'No', 'alla', '#casta', 'dei', 'partiti.', 'No', 'al', '#malaffare', '@861285225934020', '#salvinidimettiti', 'e', '#salvinifattiprocessare', 'perché', 'siamo', 'tutti', 'uguali', 'davanti', 'alla', 'legge', 'e', 'la', '#Magistratura', 'è', 'ancora', 'indipendente.', 'Perché', '#siamoumani', 'e', 'basta', '#Diciotti', '#facciamorete', 'Se', 'la', '#SeaWatch', 'fosse', 'piena', 'di', 'gattini', 'sareste', 'già', 'partiti', 'con', 'i', 'pedalò', 'per', 'salvarli.', 'Mi', 'disgustate.', '#facciamorete', '#PortiAperti', '#salvinidimettiti', '#restiamoumani', 'I

In [ ]:
split_text = convert(text_list)
allWordExceptStopDist = nltk.FreqDist(w.lower() for w in split_text if w not in stopwords) 

In [ ]:
mostCommon= allWordExceptStopDist.most_common(120)
mostCommon

[('#m5s', 7),
 ('la', 6),
 ('senza', 6),
 ('[url]', 6),
 ('dovrebbe', 4),
 ('invece', 4),
 ('#legge', 4),
 ('non', 4),
 ('no', 4),
 ('perché', 4),
 ('#conte', 4),
 ('sono', 4),
 (',', 4),
 ('in', 4),
 ('a', 4),
 ('#renzi', 4),
 ('#draghi', 4),
 ('#salvinidimettiti', 3),
 ('legge', 3),
 ('#facciamorete', 3),
 ('governo', 3),
 ('fine', 3),
 ('fare', 3),
 ('#governo', 3),
 ('#salvini', 3),
 ('@943206476637778', 3),
 ('#salvinivergognati', 3),
 ('mai', 3),
 ('.', 3),
 ('avanti', 3),
 ('essere', 3),
 ('#pd', 3),
 ('che', 3),
 ('salvini', 3),
 ('può', 3),
 ('solo', 3),
 ('@804831429960458', 3),
 ('via', 3),
 ('sempre', 3),
 ('e', 3),
 ('#speranzadimettiti', 3),
 ('#letta', 3),
 ('stare', 2),
 ('w', 2),
 ('ancora', 2),
 ('#diciotti', 2),
 ('se', 2),
 ('mi', 2),
 ('#portiaperti', 2),
 ('va', 2),
 ('mano', 2),
 ('eh', 2),
 ('niente', 2),
 ('c’è', 2),
 ('#ministro', 2),
 ('bene', 2),
 ('diritto', 2),
 ("c'è", 2),
 ('quindi', 2),
 ('problema', 2),
 ('#roma', 2),
 ('ma', 2),
 ('strada', 2),
 ('gli

In [ ]:
error

,anonymized_tweet_id,anonymized_text,label,dataset,data_type,preds
2,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,training_politics,val,0
5,848965724551157,@868843245148057 W il #m5s\n\nNo alla #casta d...,0,training_politics,val,1
7,283508205494975,@861285225934020 #salvinidimettiti e #salvinif...,0,training_politics,val,1
8,933799779508782,Se la #SeaWatch fosse piena di gattini sareste...,0,training_politics,val,1
10,229647197643357,Intanto in #Spagna il governo socialista di #S...,1,training_politics,val,0
11,733879466972060,Sono iscritto al #M5S dal 2010 e voterò NO per...,1,training_politics,val,0
23,71184870630357,#Rami eh niente c’è un #Ministro che non sta b...,0,training_politics,val,1
27,820433310798508,Se credete che la #legge sul #RevengePorn sia ...,0,training_politics,val,1
56,494786086468365,Quelli che si lamentano della puzza di urina p...,1,training_politics,val,0
63,624468506805769,@943206476637778 I processi si fanno in tribun...,0,training_politics,val,1


Patterns di errori: 

FALSE POSITIVES:

1) Frasi contenenti sentiment negativo ma verso statements razzisti (Se la #SeaWatch fosse piena di gattini sareste già partiti con i pedalò per salvarli. *Mi disgustate.* #facciamorete #PortiAperti #salvinidimettiti #restiamoumani)

FALSE NEGATIVES:

1) Soggetto implicito (Quelli che si lamentano della puzza di urina per le strade di #Roma sono gli stessi che dicono #Fateliscendere o #portiaperti ! Ma secondo loro chi cazzo è che *piscia per strada a tutte le ore, che vagano ubriachi o senza meta?* Gli alieni? #raggi #salvini #facciamorete)


